# Predicting Cancer with DNA Methylation

This notebook is a walk-through of our project submitted to the 2020 Hack for Social Good
 _Build Hackathon.

This notebook is organized the following way:
 1. Data preparation
 2. Preprocessing
 3. Model Training and Evaluation
 4. Model Deployment

It uses data from the [TCGA Project](https://www.genome.gov/Funded-Programs-Projects/Cancer-Genome-Atlas)
to create a BigQuery table. This table is then pre-processed to create our final training set.
We then go through model training and model deployment.
At the end of the notebook, you'll get a trained and deployed model on AI Prediction.

This other [repo](https://github.com/jeremy0dell/build-hackathon) uses this deployed
model to publish a React webapp on GCP and provide a user interface to interact with
the model.

## Environment setup
This notebook has been tested with Python 3.8.5. If not done already, create the `DNA_Methylation`
virtual environment using conda and the `environment.yml` file.

### Python environment

In [39]:
# Import ML libraries
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import xgboost as xgb
import plotly.graph_objects as go
from datetime import datetime
from sklearn.metrics import f1_score, plot_confusion_matrix
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Import local libraries
from feature_engineering.get_data import read_from_gcs
from feature_engineering.preprocessing import preprocessing
from configs import configs

# Import GCP libraries
from google.cloud import bigquery, storage
import google.auth

We tested this notebook with the following version of those packages
 - Pandas: 1.1.4
 - Scikit-learn: 0.23.2
 - XGBoost: 1.2.0

Check that your environment uses those versions.

In [2]:
print(f"Pandas version used: {pd.__version__}")
print(f"Scikit-learn version used: {sklearn.__version__}")
print(f"XGBoost version used: {xgb.__version__}")

Pandas version used: 1.1.2


### GCP Environment
We heavily uses GCP services on this Notebook.

Retrieve credentials to use Google Cloud's Python libraries

In [3]:
credentials, project_id = google.auth.default()
print(project_id)

gcp-nyc


In [4]:
PROJECT_ID = configs.PROJECT_ID
DATASET = configs.DATASET
BETAS_TABLE_NAME = configs.BETAS_TABLE_NAME
CPG_SITE_TABLE_NAME = configs.CPG_SITE_TABLE_NAME

## Data preparation

In this first part, we will create two bigquery tables that will used to create the
final train and test datasets.

The SQL script are located in the `sql_queries` folder.

The first BigQuery table is called `tcga_betas` is a BigQuery partitioned and clustered table
that will hold all betas values for all TCGA patients. It will be a table in a long format:
one row per betas observation.

The second BigQuery table is called `columns_to_keep` is a BigQuery table that will hold the
list of CpG site that we will keep in the final dataset.

### First table - `tcga_betas` table: a simple SQL script

This first table will be created using a simple SQL query. The query is located in
`SQL_queries/tcga_betas.txt`.

This table will have the following format:

| case_barcode | beta_value         | CpG_probe_id | aliquot_barcode              | sample_id | sample_status | row_number |
| ------------ | ------------------ | ------------ | ---------------------------- | --------- | ------------- | ---------- |
| TCGA-HC-A6AL | 0.904757283874934  | cg08616243   | TCGA-HC-A6AL-01A-11D-A30F-05 | 01        | tumor         | 8303       |
| TCGA-HC-A6AL | 0.361544714195919  | cg22309923   | TCGA-HC-A6AL-01A-11D-A30F-05 | 01        | tumor         | 8303       |
| TCGA-HC-A6AL | 0.0234784367599006 | cg26917673   | TCGA-HC-A6AL-01A-11D-A30F-05 | 01        | tumor         | 8303       |

In [5]:
SQL_query_path = 'sql_queries/tcga_betas.txt'
with open(SQL_query_path, 'r') as f:
    sql_query = ' '.join(f.readlines())

sql_query = sql_query.replace('__DATASET__', DATASET)
sql_query = sql_query.replace('__TABLE_NAME__', BETAS_TABLE_NAME)

CREATE TABLE dna_cancer_prediction.tcga_betas
 PARTITION BY RANGE_BUCKET(row_number, GENERATE_ARRAY(1, 11000, 100))
 CLUSTER BY row_number
 AS
 
 WITH
   brca_betas AS (
   SELECT
     MIM.CpG_probe_id,
     dna.case_barcode,
     dna.aliquot_barcode,
     dna.probe_id,
     dna.beta_value,
     gc.Name
   FROM
     `isb-cgc.platform_reference.methylation_annotation` gc
   JOIN
     `isb-cgc.TCGA_hg38_data_v0.DNA_Methylation` dna
   ON
     gc.Name = dna.probe_id
   JOIN
     `isb-cgc.platform_reference.GDC_hg38_methylation_annotation` MIM
   ON
     MIM.CpG_probe_id = gc.Name),
 
   participants_id AS (
   SELECT
     DISTINCT case_barcode
   FROM
     `isb-cgc.TCGA_hg38_data_v0.DNA_Methylation`),
 
   participants_id_numbered AS (
   SELECT
     case_barcode,
     ROW_NUMBER() OVER (order by case_barcode) as row_number
   FROM
     participants_id)
 
 SELECT
   A.case_barcode,
   A.beta_value,
   A.CpG_probe_id,
   A.aliquot_barcode,
   SUBSTR(A.aliquot_barcode, 14, 2) AS sample_id,


Uncomment the line below to print the SQL query

In [ ]:
# print(sql_query)

TCGA has published in BigQuery an open dataset called `isb-cgc` (documentation
[here](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/data/TCGA_top.html)).
We are using three tables from the `isb-cgc` as source tables:
  1. `isb-cgc.TCGA_hg38_data_v0.DNA_Methylation` is the main table where the DNA Methylation values
  are found.
  2. `isb-cgc.platform_reference.methylation_annotation` is used to retrieve the CpG site names
  3. `isb-cgc.platform_reference.GDC_hg38_methylation_annotation` is used to retrieve the CpG site ids.

We are creating our raw table in two parts:
 1. creating the `brca_betas` table that contains a join of those three tables mentioned above
 2. selecting columns from this `brca_betas` table and creating the `sample_status` field that will
 be used as prediction label.

To build the `sample_status`, we are using the `aliquot_barcode` field from TCGA data and more
specifically the sample part of the barcode. According to the
[documentation](https://docs.gdc.cancer.gov/Encyclopedia/pages/TCGA_Barcode/), the sample part
gives information about the label cancerous status:
 - if the sample is between 01 and 09, the tissue is cancerous
 - else if between 10 and 19, the tissue is normal
 - else it is a control sample.

We can execute the job to create the table in BigQuery.

In [7]:
client = bigquery.Client()
query_job = client.query(sql_query)

### Second table - `columns_to_keep`: using dataproc

The second table will hold the list of CpG sites that will be included in the training dataset.

Indeed, we want to keep only the 5,000 best CpG site among the 500,000 that will be able to
separate cancerous vs non-cancerous observations.

This table will be called `cpg_site_list` created in the same dataset.

This table will only have a single column and 5,000 rows

| CpG_probe_id |
| ------------ |
| cg21270593   |
| cg10107671   |
| cg11768416   |

In [ ]:
SQL_query_path = 'sql_queries/cpg_site_list.txt'
with open(SQL_query_path, 'r') as f:
    sql_query = ' '.join(f.readlines())

sql_query = sql_query.replace('__DATASET__', DATASET)
sql_query = sql_query.replace('__TCGA_BETAS_TABLE__', BETAS_TABLE_NAME)
sql_query = sql_query.replace('__CPG_SITE_TABLE_NAME__', CPG_SITE_TABLE_NAME)

Uncomment the line below to print the SQL query

In [ ]:
# print(sql_query)

We are selecting the 5,000 CpG sites that maximize the variance in-between
groups (i.e. normal and tumor patient). For this, we are applying the following
steps in order:
  1. computing the count of values per CpG sites
  2. filtering out CpG sites which are mesured in less than 90% of patients
  3. computing the average beta value per CpG sites
  4. computing the average bet value per CpG sites and per label (normal /tumor)
  5. computing the variance in-between groups
  6. selecting the best 5,000 sites

In [9]:
client = bigquery.Client()
query_job = client.query(sql_query)

Job [81fa880ca37c4901bc736ce3e941c5ea] submitted.
Waiting for job output...
build_hackathon_dnanyc_tfx_pipeline
20/10/20 14:20:36 INFO org.spark_project.jetty.util.log: Logging initialized @2728ms
20/10/20 14:20:36 INFO org.spark_project.jetty.server.Server: jetty-9.3.z-SNAPSHOT, build timestamp: unknown, git hash: unknown
20/10/20 14:20:36 INFO org.spark_project.jetty.server.Server: Started @2816ms
20/10/20 14:20:36 INFO org.spark_project.jetty.server.AbstractConnector: Started ServerConnector@e5b4927{HTTP/1.1,[http/1.1]}{0.0.0.0:4040}
20/10/20 14:20:36 WARN org.apache.spark.scheduler.FairSchedulableBuilder: Fair Scheduler configuration file not found so jobs will be scheduled in FIFO order. To use fair scheduling, configure pools in fairscheduler.xml or set spark.scheduler.allocation.file to a file that contains the configuration.
20/10/20 14:20:37 INFO org.apache.hadoop.yarn.client.RMProxy: Connecting to ResourceManager at build-hackathon-nyc-cluster-m/10.0.0.39:8032
20/10/20 14:20:

### Creating training dataset

We will now use those training tables to create our training dataset. The training dataset will be
saved into a GCS bucket defined in the `configs/configs.py` file.

For this, we are generating programmatically the SQL queries that filters
out measurements that are not part of the CpG site list.

We then create the final dataset in the `pivot_data` function using pandas.

In [10]:
DATASET_PATH = configs.RAW_DATASET_PATH
DATASET_NAME = configs.RAW_DATASET_NAME

In [17]:
def configure_gcs(gcp_project_id=PROJECT_ID):
    current_gcs_client = storage.Client(project=gcp_project_id)
    return current_gcs_client

def save_to_gcs(current_df, gcs_path, file_name):
    current_df.to_csv(gcs_path + file_name)

In [22]:
def download_from_bigquery(gcp_project_id, dataset, table, list_of_columns, start_index, end_index):
    formatted_columns = "', '".join(list_of_columns)
    query = f"""
    SELECT beta_value, CpG_probe_id, aliquot_barcode, sample_status
    from `{dataset}.{table}`
    where CpG_probe_id in ('{formatted_columns}') and row_number >= {start_index} and row_number <= {end_index}
    """
    betas_df = pd.read_gbq(query, project_id=gcp_project_id)
    return betas_df

def download_columns_to_keep(gcp_project_id, dataset, table):
    query = f"""
    SELECT CpG_probe_id
    from `{dataset}.{table}`
    """
    cpg_df = pd.read_gbq(query, project_id=gcp_project_id)
    return cpg_df['CpG_probe_id'].values

def merge_and_pivot_by_slices(gcp_project_id, dataset, table, cpg_list, start_index, end_index):
    df_betas = download_from_bigquery(gcp_project_id, dataset, table, cpg_list, start_index, end_index)
    df_p = df_betas.pivot(index="aliquot_barcode", columns='CpG_probe_id', values='beta_value').reset_index()
    df_labels = df_betas[['aliquot_barcode', 'sample_status']].drop_duplicates()
    df_final = df_p.merge(df_labels,
                          how='left', on='aliquot_barcode')
    df_final = df_final.set_index('aliquot_barcode')
    # Reorder columns
    df_final = df_final[columns]
    return df_final

def pivot_data(cpg_project_id, dataset, betas_table, cpg_site_list_table, max_index=11000, steps=1000):
    cpg_list = download_columns_to_keep(cpg_project_id, dataset, cpg_site_list_table)
    df_final = merge_and_pivot_by_slices(cpg_project_id, dataset, betas_table, cpg_list, 1, steps)
    start_index = steps + 1
    for i in range(start_index, max_index, steps):
        print(f"Processing index {i} to {i + steps}")
        new_df = merge_and_pivot_by_slices(cpg_project_id, dataset, betas_table, columns, i, i + steps)
        df_final = df_final.append(new_df)
    return df_final

In [ ]:
gcs_client = configure_gcs(PROJECT_ID)
columns = download_columns_to_keep(PROJECT_ID, DATASET, 'cpg_site_list')
df = pivot_data(PROJECT_ID, DATASET, 'tcga_betas', 'cpg_site_list')
save_to_gcs(df, DATASET_PATH, DATASET_NAME)

# Model Training

Using the saved dataset in Google Cloud Storage, we will now preprocess the dataset
to create a train and test split, train our model on the training set and output
and evaluation metric on the evaluation set.

We chose to an XGBoost model as part of this project. It's a model that first gives
a very good accuracy on complex problems. The model is also easy to interpret
as it is a tree based model.

## Data preprocessing

Read data from Google Cloud Storage

In [5]:
BUCKET_NAME = configs.GCS_BUCKET
RAW_DATASET_PATH = configs.RAW_DATASET_PATH
LABELS = configs.RAW_LABEL_NAME
INDEX = configs.RAW_INDEX_NAME
betas, labels, cpg_sites, index = read_from_gcs(BUCKET_NAME, RAW_DATASET_PATH, INDEX, LABELS)

Blob training_data/tcga-binary.csv downloaded to training_data.csv.


Map classes to a binary integer classification. 1 will represent a tumorous observation and 0 will represent a normal
observation

In [6]:
map_labels_to_classes = {
  'tumor': 1,
  'normal': 0
}
labels = [map_labels_to_classes[elt] for elt in labels]

Apply the final preprocessing steps before starting training. In order, we will:
 1. Drop columns that have more than 10% of values missing
 2. Drop rows that have more than 10% of values missing
 3. Fill remaining NA values using a `KNNImputer`
 4. Split into a train / test dataset using 70% of data for train and 30% for the test
 5. Standardize the train and test sets (bringing the mean and std to 0 and 1 by columns)
 6. Balance observation in the train dataset. To get best results, we need to add
 negative (normal) observations to our dataset to get to at least 30% negative rows.
 This is done by SMOTE (*Synthetic Minority Oversample Technique*).

In [8]:
X_train, X_test, y_train, y_test, labels, cpg_sites = preprocessing(betas, labels, cpg_sites, index, smote=True,
                                                                   fill_na_strategy='knn', sampling_strategy=0.3)

=== Drop Columns and Rows ===
Dropping 0 because of missing labels
New Shape = (12298, 5000)
Dropping columns which have more than 10% of values missing
0 columns will be dropped.
betas: New shape is (12298, 5000)
cpg_sites: New shape is (5000,)

Dropping rows which have more than 10% of values missing
We will drop 0 rows
betas: New shape is (12298, 5000)
labels: New shape is (12298,)

=== Fill remaining NAs ===
Filling remaining NA values using a KNNImputer
38743 NA were filled, i.e. approximately 3.15 per rows

=== Train / Test Split ===
Splitting dataset into train and test
Train = 70 %
Test = 30 %

=== Standardize dataset ===
The average of column mean on train is 0.00
The average of column mean on test is 0.01

=== Balance dataset with oversample ===
[(0, 786), (1, 7822)]
The resampling_strategy gives the following repartition {0: 2346, 1: 7822}
1560 rows were added in the training data


We can verify that we now have at least 30% of negative observations.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=y_train))

# Overlay both histograms
fig.update_layout(bargroupgap=0.2,
                  title="Histogram of training labels after re-balancing the dataset",
                  xaxis_title="Cancerous or normal observations",
                  yaxis_title="Count of observations")
fig.update_layout(
    margin=dict(
        l=40,
        r=40,
        b=40,
        t=40,
        pad=4
    ),
    title={
        'x':0.5})
fig.show()


## Training

Now that we have chosen XGBoost, we need to find the hyper-parameters that will provide the best accuracy on
our dataset.
To do this, we are defining a Stratified-KFold cross-validation. Stratified means that we are keeping the same
proportion of positive and negative observations in each validation fold.
Here, we choose to create 4 folds. This means that we will do four fits: each time, training on 3/4 of the training
data while testing on the remaining third.
After all these fits, we average the F1 score we got and take the hyper-parameter combination that gives the best
average F1 score.

Let's explain the XGBoost parameters we are using:
 - `objective=binary:logistic`: this is a binary classification problem, and we want to output the probability that
   an observation is cancerous
 - `colsample_bytree=0.8`: we are considering only 80% of features to build each tree. This prevents over-fitting
 - `learning_rate=0.1`: prevents over-fitting by shrinking the feature weights
 - `subsample=0.8`: we are considering only 80% of the observations to build each tree. This prevents over-fitting.
 - `nthread=4`: uses 4 different thread to train the model. Enables parallel processing.

In our parameter grid, we are testing the following parameters:
 - `max_depth`: the maximum depth of each tree in the model. The higher the parameter, the higher the variance
 - `min_child_weight`: the minimum sum of hessian weight needed to create a child node and partition further the tree.
   The higher the parameter, the lower the variance.
 - `gamma`: the minimum loss reduction required to make a further partition on a leaf node. The higher the parameter,
   the lower the variance.
 - `alpha`: L1 regularisation term on weights. The higher the parameter, the lower the variance.
 - `n_estimator`: number of trees in the model. The higher the parameter, the higher the variance.

See [here](https://xgboost.readthedocs.io/en/latest/parameter.html) for the complete XGBoost documentation.

In [ ]:
# Define the KFolds
kf = StratifiedKFold(n_splits=4)
# Define the parameter grid to test
param_test1 = {
    'max_depth': range(3,10,2),
    'min_child_weight': range(1,6,2),
    'gamma':[i/10.0 for i in range(0,5)],
    'alpha':[6,8,10,12],
    'n_estimators': [1e2, 1e3, 1e4]
}
estimator = xgb.XGBClassifier(objective='binary:logistic',
                              colsample_bytree=0.8,
                              learning_rate=0.1,
                              subsample=0.8,
                              nthread=4)

grid_search = GridSearchCV(estimator=estimator, param_grid=param_test1,
                           scoring='f1_weighted',n_jobs=4, cv=5)
# Train the model
grid_search.fit(X_train, y_train)

Let's retrieve the best parameters from the GridSearch optimizer.

In [ ]:
best_params = grid_search.best_params_
best_max_depth = best_params['max_depth']
best_min_child_weight = best_params['min_child_weight']
best_gamma = best_params['gamma']
best_alpha = best_params['alpha']
best_n_estimators = best_params['n_estimators']


Now, we can fit the model with the optimized hyper-parameters on the entire dataset.
That will be our final model for this notebook.

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic',
                           colsample_bytree=0.8,
                           learning_rate=0.1,
                           subsample=0.8,
                           nthread=4,
                           max_depth=10,
                           gamma=0.1,
                           alpha=6,
                           n_estimators=100)
model.fit(X_train, y_train)

## Evaluation

Let's now analyze the model performance

### F1 Score

Let's compute the f1 score as a rough proxy for model performance. The F1 is the
harmonic mean between precision and recall.

$$ \text{F1 Score} = 2 \frac{\text{precision}\cdot \text{recall}}{\text{precision} + \text{recall}}$$

First, on the training set and then on the test set.

In [ ]:
# Training
training_predicted_labels = model.predict(X_train)
f1_training = f1_score(training_predicted_labels, y_train)
print(f"The train accuracy is {f1_training:.3f}")

# Testing
testing_predicted_labels = model.predict(X_test)
f1_testing = f1_score(testing_predicted_labels, y_test)
print(f"The test accuracy is {f1_testing:.3f}")

## Confusion Matrix

To go one level deeper, we can output the confusion matrix on both the training set and the test set.

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 10))
f.suptitle("Confusion matrix on the *training* set")
plot_confusion_matrix(model, X_train, y_train, ax=ax, values_format='.0f', normalize=None);

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 10))
f.suptitle("Confusion matrix on the test set")
plot_confusion_matrix(model, X_test, y_test, ax=ax, values_format='.0f', normalize=None)
f.tight_layout()
f.savefig("confusion_matrix.png", dpi=200)

# Training and deploying models on GCP

We have trained our XGBoost model locally on this notebook. But, in a production example
we would like to better optimize the hyper-parameter by giving more options. We will do
this by using the AI Platform Training service.
Once this is done, we also need to deploy our model into production. That will be done
by the AI Platform Prediction service.

## AI Platform Training

We will first save to GCS the train and test datasets we used earlier

In [ ]:
train_dataset_path = configs.TRAIN_DATASET_PATH
test_dataset_path = configs.TEST_DATASET_PATH
training_label_name = configs.TRAINING_LABEL_NAME

In [ ]:
train_df = pd.DataFrame(X_train, columns=cpg_sites)
train_df[training_label_name] = y_train
save_to_gcs(train_df, f"gs://{BUCKET_NAME}/", train_dataset_path)

In [ ]:
test_df = pd.DataFrame(X_test, columns=cpg_sites)
test_df[training_label_name] = y_test
save_to_gcs(test_df, f"gs://{BUCKET_NAME}/", test_dataset_path)

To start an AI Platform training job, we will first create a Docker image of our code and then use the gcloud command line to start the job

In [36]:
IMAGE_REPO_NAME = configs.IMAGE_REPO_NAME
IMAGE_TAG = configs.IMAGE_TAG
IMAGE_URI = configs.IMAGE_URI
TRAINING_DOCKERFILE = configs.TRAINING_DOCKERFILE

In [37]:
!docker build -f {TRAINING_DOCKERFILE} -t {IMAGE_URI} ./

Sending build context to Docker daemon   1.14GB
Step 1/12 : FROM python:3.8.6
 ---> f5e423f5ce1f
Step 2/12 : WORKDIR /root
 ---> Using cache
 ---> 2b02197714a0
Step 3/12 : RUN pip install xgboost==1.2.1 scikit-learn==0.23.2 pandas==1.1.3 joblib==0.17.0
 ---> Using cache
 ---> e7d979897e13
Step 4/12 : RUN pip install cloudml-hypertune
 ---> Using cache
 ---> 0bda83cd5ccf
Step 5/12 : RUN pip install google-cloud-storage==1.32.0
 ---> Using cache
 ---> 3d3ee957411a
Step 6/12 : RUN wget -nv     https://dl.google.com/dl/cloudsdk/release/google-cloud-sdk.tar.gz &&     mkdir /root/tools &&     tar xvzf google-cloud-sdk.tar.gz -C /root/tools &&     rm google-cloud-sdk.tar.gz &&     /root/tools/google-cloud-sdk/install.sh --usage-reporting=false         --path-update=false --bash-completion=false         --disable-installation-options &&     rm -rf /root/.config/* &&     ln -s /root/.config /config &&     rm -rf /root/tools/google-cloud-sdk/.install/.backup
 ---> Using cache
 ---> a385905dd329


Then, we push this image to Google Cloud Container Registry

In [38]:
!docker push {IMAGE_URI}

The push refers to repository [gcr.io/gcp-nyc/gcr.io/gcp-nyc/dna-methylation-cancer]

4a351004: Preparing 
5396cbe7: Preparing 
f1f5e1c8: Preparing 
bcc978e3: Preparing 
0dd0e47b: Preparing 
53a9bd82: Preparing 
8930cc4a: Preparing 
261fba2e: Preparing 
c05f6425: Preparing 
863eb588: Preparing 
381add18: Preparing 
43721c9b: Preparing 
c05f6425: Waiting g 
53a9bd82: Waiting g 
2cfe5c83: Preparing 
4f1da707: Preparing 
a351004: Pushed lready exists kBBxgboost-binary: digest: sha256:6666a8141f527a54788fb71c35289bd5b2cc8443434a77cb8b732c3b0cee0f4d size: 3894


Finally, we can push the job to AI Platform

In [44]:
date = datetime.now().strftime("%Y%m%d_%H%M%S")
JOB_NAME = f'dna_methylation_xgboost_binary_{date}'
MODEL_DIR= f"gs://{BUCKET_NAME}/job_dir/{date}"
REGION = configs.REGION
HPTUNING_CONFIG = configs.HPTUNING_CONFIG

In [48]:
!gcloud ai-platform jobs submit training {JOB_NAME} --region {REGION} --master-image-uri {IMAGE_URI} --config {HPTUNING_CONFIG} -- --model-dir={MODEL_DIR}

Job [dna_methylation_xgboost_binary_20201021_212655] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe dna_methylation_xgboost_binary_20201021_212655

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs dna_methylation_xgboost_binary_20201021_212655
jobId: dna_methylation_xgboost_binary_20201021_212655
state: QUEUED


## AI Platform Prediction

The final step will be to deploy our model into AI Platform Prediction
to be able to use it in production.

For this, we are packaging the code inside `ai-platform-prediction`
and saving it to GCS.

In [ ]:
!cd ai-platform-prediction/prediction_routing/ && python setup.py sdist --format=gztar

In [ ]:
MODEL_NAME=configs.MODEL_NAME
VERSION_NAME=configs.VERSION_NAME
ORIGIN='gs://dna-methylation-cancer/job_dir/20201021_204518/'
PACKAGE_URIS=configs.PACKAGE_URIS
PREDICTION_CLASS=configs.PREDICTION_CLASS
RUNTIME_VERSION=configs.RUNTIME_VERSION
PYTHON_VERSION=configs.PYTHON_VERSION

In [ ]:
!gsutil cp ai-platform-prediction/prediction_routing/dist/my_custom_code-0.2.tar.gz {PACKAGE_URIS}

Create a model on AI Platform Prediction

In [ ]:
!gcloud ai-platform models create {MODEL_NAME} --regions 'us-central1'

In [ ]:
!gcloud beta ai-platform versions create {VERSION_NAME} --model {MODEL_NAME} --runtime-version {RUNTIME_VERSION} --python-version {PYTHON_VERSION} --origin {ORIGIN} --package-uris {PACKAGE_URIS} --prediction-class {PREDICTION_CLASS}


In [78]:
!cd ai-platform-prediction/prediction_routing/ && python setup.py sdist --format=gztar

running sdist
running egg_info
writing my_custom_code.egg-info/PKG-INFO
writing dependency_links to my_custom_code.egg-info/dependency_links.txt
writing top-level names to my_custom_code.egg-info/top_level.txt
reading manifest file 'my_custom_code.egg-info/SOURCES.txt'
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'

running check


creating my_custom_code-0.2
creating my_custom_code-0.2/my_custom_code.egg-info
copying files to my_custom_code-0.2...
copying predictor.py -> my_custom_code-0.2
copying setup.py -> my_custom_code-0.2
copying my_custom_code.egg-info/PKG-INFO -> my_custom_code-0.2/my_custom_code.egg-info
copying my_custom_code.egg-info/SOURCES.txt -> my_custom_code-0.2/my_custom_code.egg-info
copying my_custom_code.egg-info/dependency_links.txt -> my_custom_code-0.2/my_custom_code.egg-info
copying my_custom_code.egg-info/top_level.txt -> my_custom_code-0.2/my_custom_code.egg-info
Writing my_custom_code-0.2/setup.cfg
Creating tar archive
removing 'my_custom_code-0

In [85]:
!gsutil cp ai-platform-prediction/prediction_routing/dist/my_custom_code-0.2.tar.gz {PACKAGE_URIS}

Copying file://ai-platform-prediction/prediction_routing/dist/my_custom_code-0.2.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.                                      


In [75]:
!gcloud ai-platform models create {MODEL_NAME} --regions 'us-central1'

Using endpoint [https://ml.googleapis.com/]
Created ml engine model [projects/gcp-nyc/models/dna_cancer_prediction].


In [91]:
!gcloud beta ai-platform versions create {VERSION_NAME} --model {MODEL_NAME} --runtime-version {RUNTIME_VERSION} --python-version {PYTHON_VERSION} --origin {ORIGIN} --package-uris {PACKAGE_URIS} --prediction-class {PREDICTION_CLASS}


Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    
